# 환경설정

In [1]:
import os
import gc
import copy
import numpy as np
import pandas as pd

import easydict
from tqdm import notebook
from tqdm import tqdm

# 이미지
from PIL import Image
import cv2
# 시각화
import matplotlib.pyplot as plt
import matplotlib as mpl

# PyTorch
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW

from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision import transforms
import timm

# 경로 설정 & 라벨링

In [6]:
train_dir = '/opt/ml/input/data/train/'
train = pd.read_csv(os.path.join(train_dir, 'train.csv'))
image_dir = os.path.join(train_dir, 'images/')

# train_df 만들기 / 라벨링

labels = []

for i in train.index:
    gender = 0
    # 성별에 따른 분류
    if train.loc[i]['gender'] == 'female':
        gender = 3

    # 나이에 따른 분류
    age = 0
    if 30 <= train.loc[i]['age'] < 60:
        age += 1
    elif 60 <= train.loc[i]['age']:
        age += 2


    # 마스크 착용에 따른 분류
    for dirpath, dirnames, filenames in os.walk(image_dir+train.loc[i]['path']):
        for filename in filenames:
            # 마스크 제대로 쓴 경우 (5장)
            if filename[0] == 'm':
                label = {"img_path": dirpath+"/"+filename,
                        "label": gender+age}
                labels.append(label)

            # 마스크 이상하게 착용(코, 턱스크) (1장)
            elif filename[0] == 'i':
                label = {"img_path": dirpath+"/"+filename,
                        "label": 6+gender+age}
                labels.append(label)
            # 미착용 (1장)
            elif filename[0] == 'n':
                label = {"img_path": dirpath+"/"+filename,
                        "label": 12+gender+age}
                labels.append(label)
            
train_df = pd.DataFrame(labels)
train_df = train_df.sort_values(['label', 'img_path'])
train_df = train_df.reset_index(drop=True)
train_df.head()

,img_path,label
0,/opt/ml/input/data/train/images/000309_male_As...,0
1,/opt/ml/input/data/train/images/000309_male_As...,0
2,/opt/ml/input/data/train/images/000309_male_As...,0
3,/opt/ml/input/data/train/images/000309_male_As...,0
4,/opt/ml/input/data/train/images/000309_male_As...,0


In [8]:
len(train_df)

18900

In [9]:
train_df.groupby('label').count()

,img_path
label,
0,2745
1,2050
2,415
3,3660
4,4085
5,545
6,549
7,410
8,83


# Dataset & DataLoader

In [10]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        data = self.img_paths.iloc[index]

        img_path = data['img_path']
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        label = data['label']
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [23]:
transform = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])])
                                                
dataset = TrainDataset(train_df, transform)

# Hyper Parameter

In [24]:
import random
batch_size = 32
lr = 1e-4
random_seed = 2021
random.seed(random_seed)
torch.manual_seed(random_seed)

# train / valid 나누기

In [25]:
## data split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=random_seed)
datasets = {}
datasets['train'] = Subset(dataset, train_idx)
datasets['valid'] = Subset(dataset, val_idx)

## data loader 선언
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=4)
dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=4)

batch_num['train'], batch_num['valid'] = len(dataloaders['train']), len(dataloaders['valid'])
print('batch_size : %d,  tvt : %d / %d' % (batch_size, batch_num['train'], batch_num['valid']))

batch_size : 32,  tvt : 574 / 144


# 모델링

In [30]:
model_name = "swin_large_patch4_window7_224"
model = timm.create_model("swin_large_patch4_window7_224", pretrained=True, num_classes=18)

# 훈련

In [31]:
import time
from sklearn.metrics import f1_score
fold = 1
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    global version
    global fold
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_f1 = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)


        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            epoch_f1 = 0.0
            n_iter = 0.0          
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
                n_iter += 1
                num_cnt += len(labels)
            if phase == 'train':
                scheduler.step()
            
            epoch_fl = epoch_f1 / n_iter
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f} F1: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_fl))
           
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_f1 = epoch_f1

                best_model_wts = copy.deepcopy(model.state_dict())
#                 best_model_wts = copy.deepcopy(model.module.state_dict())
                print('==> best model saved - %d / %.1f / %.4f'%(best_idx, best_acc, best_f1))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))
    print('Best valid f1: %d - %.1f' %(best_idx, best_f1))
    # load best model weights
    model.load_state_dict(best_model_wts)
    save_name =  f"0831Fold_{fold}_{model_name}_batch{batch_size}_lr{lr}_{round(best_acc,2)}.pt"
    fold += 1
    torch.save(model, save_name)

    print('model saved')
    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc

In [32]:
from sklearn.utils import class_weight
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # set gpu

model = model.to(device)

weights = class_weight.compute_class_weight('balanced',
                                            np.unique(train_df['label']),
                                            train_df['label'])
weights = torch.FloatTensor(weights).to(device)

#criterion = nn.CrossEntropyLoss(weight=weights)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.AdamW(model.parameters(), lr=lr)

exp_lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer_ft,
                                          T_max=10,
                                          eta_min=0)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17], y=0         0
1         0
2         0
3         0
4         0
         ..
22927    17
22928    17
22929    17
22930    17
22931    17
Name: label, Length: 22932, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [ ]:
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, dataloaders, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)

In [33]:
start_model_wts = copy.deepcopy(model.state_dict())

In [34]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5)
k = 1
train_label = train_df['label'].values
for train_idx, val_idx in kfold.split(train_df, train_label):
    model.load_state_dict(start_model_wts)

    print('\n')
    print('='* 15 +f'{k}-Fold Cross Validation'+'=' * 15)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['valid'] = Subset(dataset, val_idx)
    dataloaders= {}
    dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=4)
    dataloaders['valid'] = torch.utils.data.DataLoader(datasets['valid'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=4)
    model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, dataloaders, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]



===============1-Fold Cross Validation===============
Epoch 0/19
----------
train Loss: 0.34 Acc: 88.6 F1: 0.8241


  5%|▌         | 1/20 [08:02<2:32:48, 482.55s/it]

valid Loss: 0.47 Acc: 85.1 F1: 0.5538
==> best model saved - 0 / 85.1 / 79.7405
Epoch 1/19
----------
train Loss: 0.08 Acc: 97.4 F1: 0.9554


 10%|█         | 2/20 [16:05<2:24:48, 482.68s/it]

...

 95%|█████████▌| 19/20 [2:33:18<08:05, 485.02s/it]

valid Loss: 1.01 Acc: 82.4 F1: 0.6131
Epoch 19/19
----------
train Loss: 0.02 Acc: 99.4 F1: 0.9898


100%|██████████| 20/20 [2:41:22<00:00, 484.14s/it]

valid Loss: 1.33 Acc: 77.2 F1: 0.5498
Training complete in 161m 23s
Best valid Acc: 14 - 86.1
Best valid f1: 14 - 89.8


model saved


In [2]:
import torch
torch.cuda.empty_cache()